# Recursion
Recursion in computer science is a method where the solution to a problem depends on solutions to smaller instances of the same problem (as opposed to iteration).  
  
Each recursive algorithm consists of:
- **Base case**: a case for which the answer is _known_ (and can be expressed _without_ recursion).
- **Recursive (general) case**: a case for which a solution is expressed in terms of a _smaller version of itself_.

Each recursive algorithm **must** have at least one base case. If the base case is not reached or not provided, then the program would run out of stack space (stack overflow).

![Russian Dols](img/russian-dols.jpg)

### Example: sum all integers from $[0..n]$ without using loops
Recurrence formula:  
$F(n)=\begin{cases}
    0 & \text{if $n=0$ (base case)}.\\
    F(n-1) + n & \text{if $n>0$}.
  \end{cases}$

In [ ]:
def sum_recursive(n):
    if n == 0:
        return 0
    
    return n + sum_recursive(n - 1)

In [ ]:
def sum_and_print(n):
    print(f'Sum from 1 to {n} is {sum_recursive(n)}')

In [ ]:
sum_and_print(30)

In [ ]:
sum_and_print(300)

In [ ]:
sum_and_print(3000)

Uh-oh, we run out of stack space! Let's fix it by increasing the stack size.

In [ ]:
import sys
sys.setrecursionlimit(10000) # this is a bad idea in general

In [ ]:
sum_and_print(3000)

## Pros and Cons of Recursion
**Good:**
- Recursive algorithms are more **elegant** and **easier** to understand.
- Recursion is prefered in **functional languages** (with **tail recursion optimization** leading to little overhead).

**Bad:**
- Recursive algorithms might take **more memory** than iterative algorithms.
- Recursive algorithms might be **slower** than iterative algorithms.
- Recursive algorithms can cause running out of stack space (**stack overflow**).
  
### _IMPORTANT: recursion requires additional memory proportional to the depth of the call stack_


In [ ]:
from utils import measure_call
input_size = 5000

In [ ]:
def sum_iterative(n):
    total = 0
    k = 0
    while k <= n:
        total += k
        k += 1
        
    return total

In [ ]:
measure_call(lambda: sum_recursive(input_size), 'Sum recursive')

In [ ]:
measure_call(lambda: sum_iterative(input_size), 'Sum iterative')

## Tail Recursion Optimization

Tail-recursive functions are functions in which **all recursive calls are tail calls** and hence do not build up any deferred operations. With a compiler or interpreter that treats tail-recursive calls as **jumps rather than function calls**, a tail-recursive function **will execute using constant space**. Thus the program is essentially iterative, equivalent to using imperative language control structures like the `for` and `while` loops.

### Example with tail recursion: Greatest Common Divisor

In [ ]:
def gcd(x, y):
    if y == 0:
        return x
    else:
        return gcd(y, x % y)

### Example without tail recursion: Factorial

In [ ]:
def factorial(n):
    if n <= 1:
        return 1
    
    return n * factorial(n - 1)

### Languages with Tail Recursion Optimization
- **Scala**: by default
- **Kotlin**: includes `tailrec` modifier for functions
- **JavaScript (ES 6.0)**: implemented in Safari/WebKit
- **C/C++/Objective-C/Swift**: compiler optimizes tail calls when -O1 (or higher) option specified but it is easily disturbed by calls added by ARC.

### Languages without Tail Recursion Optimization
- **Java**
- **C#**
- **Python**
<br><img style="float: left;" src="img/python-tail-recursion.jpg">


# Dynamic Programming

A problem can be solved with dynamic programming if it has the following properties:
- **Optimal substructure**  
    To solve the original problem of size $n$, we solve **smaller** problems of the same type. An **optimal** solutions to a problem incorporate **optimal solutions to related subproblems**, which we may solve independently.
- **Overlapping Subproblems**  
    Solutions of the subproblems are reused.

## Example: Finding $n$-th Fibonacci Number
### Recurrence definition:  
$F(n)=\begin{cases}
    0, & \text{if $n=1$}.\\
    1, & \text{if $n=2$}.\\
    F(n-1) + F(n-2), & \text{otherwise}.
  \end{cases}$

In [ ]:
def fibonacci(n):
    if n==1:
        return 0
    if n==2:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

**Time complexity:** $\Theta(2^n)$ - total **number of nodes**  
**Space complexity:** $\Theta(n)$ - **height** of the recursion tree

<br><img style="float: left; width: 800px;" src="img/fib-tree.png">

In [ ]:
fibonacci(10)

## Time-memory trade-off
- **Dynamic programming uses additional memory to save computation time**.  
    Each subproblem is only solved once. Its solution is saved so we can look it up instead of recomputing.
- **An _exponential-time_ solution may be transformed into a _polynomial-time_ solution**.  
    A dynamic-programming approach runs in polynomial time when the number of _distinct_ subproblems involved is polynomial in the input size and we can solve each such subproblem in polynomial time.

# Exponential vs Polynomial
### Exponential = Slow
### Polynomial = Fast
<br><img style="float: left;" src="img/lenny-white-carl-black.jpg">

## Methods

### Top-down (memoization)
**Start with bigger problem**: In this approach, we write the procedure recursively in a natural manner, but modified to **save the result of each subproblem** (usually in an array or hash table). The procedure now **first checks to see whether it has previously solved this subproblem**. If so, **it returns the saved value**, saving further computation at this level; if not, the procedure computes the value in the usual manner. We say that the recursive procedure has been **memoized**; it “remembers” what results it has computed previously.

In [ ]:
def fibonacci_top_down(n, memo={}):
    if n in memo:
        return memo[n]
    
    if n==1:
        memo[n] = 0
    elif n==2:
        memo[n] = 1
    else:
        memo[n] = fibonacci_top_down(n-1, memo) + fibonacci_top_down(n-2, memo)
        
    return memo[n]

In [ ]:
fibonacci_top_down(10)

**Time complexity:** $\Theta(n)$ - number of **unique** subproblems  
**Space complexity:** $\Theta(n)$ - **height** of the recursion tree + **additional** memory for solution storage
<br><img style="float: left; width: 500px;" src="img/memoization.png">

In [ ]:
from utils import fibonacci_top_down_print_stack
fibonacci_top_down_print_stack(10)

### Bottom-up (tabulation)
**Start with smaller problems**: This approach typically depends on some natural notion of the “size” of a subproblem, such that **solving any particular subproblem depends only on solving “smaller” subproblems**. We **sort** the subproblems by size and solve them in size order, **smallest first**. When solving a particular subproblem, we have already **solved all of the smaller subproblems** its solution depends upon, and we have saved their solutions. We solve each subproblem only once, and when we first see it, we have already solved all of its prerequisite subproblems.

In [ ]:
def fibonacci_buttom_up(n):
    if n==1:
        return 0
    if n==2:
        return 1
    
    dp = [None] * (n + 1)
    dp[1] = 0
    dp[2] = 1
    
    k = 3
    while k <= n:
        dp[k] = dp[k - 1] + dp[k - 2]
        k += 1
        
    return dp[n]

**Time complexity:** $\Theta(n)$ - number of **iterations** in `while` loop  
**Space complexity:** $\Theta(n)$ - **additional** memory for solution storage

In [ ]:
fibonacci_buttom_up(10)

In [ ]:
from utils import fibonacci_buttom_up_print_dp
fibonacci_buttom_up_print_dp(10)

### Top-down Space optimization
We only need `2` previous solutions, so why to store all of them?

In [ ]:
def fibonacci_buttom_up_optimized(n):
    if n==1:
        return 0
    if n==2:
        return 1
    
    f2, f1 = 1, 0
    
    k = 3
    while k <= n:
        f2, f1 = f1, f1 + f2
        k += 1
        
    return f1

**Time complexity:** $\Theta(n)$ - number of **iterations** in `while` loop  
**Space complexity:** $\Theta(1)$ - only **constant** amount of memory required

## Relative performance

In [ ]:
from utils import measure_call

def run_trial(input_size, include_exponential=True):
    if include_exponential:
        measure_call(lambda: fibonacci(input_size), 'Fibonacci non-optimized')
    measure_call(lambda: fibonacci_top_down(input_size), 'Fibonacci top-down')
    measure_call(lambda: fibonacci_buttom_up(input_size), 'Fibonacci bottom-up')
    measure_call(lambda: fibonacci_buttom_up_optimized(input_size), 'Fibonacci bottom-up (opt)')

In [ ]:
run_trial(10)

In [ ]:
run_trial(20)

In [ ]:
run_trial(30)

In [ ]:
run_trial(40)

In [ ]:
run_trial(3000, include_exponential=False)

In [ ]:
run_trial(100000, include_exponential=False)

In [ ]:
run_trial(500000, include_exponential=False)

## 1-0 Knapsack Problem
You have a knapsack which can hold max weight $W$. You also have $n$ items, each of them has value $v_i$ and weight $w_i$. What the maximum value you can put into the knapsack?

<br><img style="float: left;" src="img/knapsack.png"><br>

For every candidate item we can choose if we take it or discard it. If we take an item the knapsack's "available" weight is reduced by the weight of the item and the total content value is increased by the value of the item. If an item is discarded - the knapsack's "available" weight and the total content value remain the same. In both cases we have one less item to consider.

$W$ - max weight for knapsack.  
$n$ - number of items.  
$w_n$ - weight of $n^{th}$ item.  
$v_n$ - value of $n^{th}$ item.  

### Base case:
$F(0, n) = 0$  
$F(W, 0) = 0$  

### Recursive case:
$F(W, n)=\max\begin{cases}
    F(W - w_n, n - 1) + v_n, & \text{take item if $W - w_n \ge 0$}.\\
    F(W, n - 1), & \text{discard item}.
  \end{cases}$

In [ ]:
def knapsack(W, n, weights, values):
    # base case
    if W == 0 or n == 0:
        return 0

    # take the item
    take_item_value = 0
    if W - weights[n - 1] >= 0:  # can we fit the item?
        take_item_value = knapsack(W - weights[n - 1], n - 1, weights, values) + values[n - 1]

    # discard the item
    discard_item_value = knapsack(W, n - 1, weights, values)

    # max of take/discard
    return max(take_item_value, discard_item_value)

Total knapsack value if you **take** $n^{th}$ item:  
`take_item_value = knapsack(W - weights[n - 1], n - 1, weights, values) + values[n - 1]`  

Total knapsack value if you **discard** $n^{th}$ item:  
`discard_item_value = knapsack(W, n - 1, weights, values)`  

Solution is the **max** between two:  
`max(take_item_value, discard_item_value)`

In [ ]:
W = 5
weights = [2, 3, 4]
values = [10.00, 3.50, 20.0]

In [ ]:
knapsack(W, 3, weights, values)

In [ ]:
def knapsack_top_down(W, n, weights, values, memo=None):
    # allocate memo[W + 1, n + 1]
    if memo is None:
        memo = create_memo(W, n)

    # lookup solution first
    if memo[W][n] is not None:
        return memo[W][n]

    # take the item
    take_item_value = 0
    if W - weights[n - 1] >= 0:  # can we fit the item?
        take_item_value = knapsack_top_down(W - weights[n - 1], n - 1, weights, values, memo) + values[n - 1]

    # discart the item
    discard_item_value = knapsack_top_down(W, n - 1, weights, values, memo)

    # max of take/discard
    memo[W][n] = max(take_item_value, discard_item_value)
    return memo[W][n]


def create_memo(W, n):
    memo = [[None] * (n + 1) for _ in range(W + 1)]

    # base case F(W, 0) = 0
    for w in range(W + 1):
        memo[w][0] = 0

    # base case F(0, n) = 0
    for n in range(n + 1):
        memo[0][n] = 0

    return memo

In [ ]:
knapsack_top_down(W, 3, weights, values)

In [ ]:
def knapsack_bottom_up(W, n, weights, values, memo=None):
    # create lookup table
    dp = create_lookup_table(W, n)

    for w in range(1, W + 1):
        for k in range(1, n + 1):
            # take the item
            take_item_value = 0
            if w - weights[k - 1] >= 0:  # can we fit the item?
                take_item_value = dp[w - weights[k - 1]][k - 1] + values[k - 1]

            # discard the item
            discard_item_value = dp[w][k - 1]

            # max of take/discard
            dp[w][k] = max(take_item_value, discard_item_value)

    return dp[W][n]


def create_lookup_table(W, n):
    dp = [[None] * (n + 1) for _ in range(W + 1)]

    # base case F(W, 0) = 0
    for w in range(W + 1):
        dp[w][0] = 0

    # base case F(0, n) = 0
    for n in range(n + 1):
        dp[0][n] = 0

    return dp

In [ ]:
knapsack_bottom_up(W, 3, weights, values)

# Before you go!
http://codingbat.com/java/Recursion-1  
http://codingbat.com/java/Recursion-2